# Project Overview: Predicting Loan Defaults

**Executive Summary**  
This project aims to develop a machine learning model to predict whether the customers of a financial institution will default on a loan based on data from their loan application. Accurately identifying potential defaulters enables financial institutions to make informed lending decisions, adjust loan terms, and better manage financial risk.

**Problem**  
Predicting loan defaults is a challenging task due to the multitude of influencing factors such as customers' demographic, financial, location, and behavioral attributes. Traditional default prediction models often oversimplify complex relationships between customer features and default risk. Machine learning offers enhanced predictive capability by capturing non-linear patterns and intricate dependencies in loan application data, enabling more accurate predictions of loan default risk.

**Objectives**  
The project aims to:
- Develop a machine learning model to predict loan defaults using customer data from loan applications.
- Compare multiple models (e.g., Logistic Regression, Random Forest, XGBoost) using the Area Under the Precision-Recall Curve (AUC-PR) as the primary evaluation metric.
- Identify key factors influencing loan default risk through feature importance analysis.

**Value Proposition**  
This project enables financial institutions to reduce loan default rates and make better and faster lending decisions by leveraging machine learning for automated and improved risk assessment. 

**Business Goals**  
The project aims to achieve the following business goals:
- Reduce losses by 100,000-200,000$ within 12 months after model deployment by decreasing the loan default rate by 10%-20%.
- Decrease loan processing time by 25%-40% by automating risk assessment, leading to less time spent on manual evaluations.
- Achieve 100% regulatory compliance with fair lending practices.

**Data**  
The dataset contains information provided by customers of a financial institution during the loan application process. It is sourced from the "Loan Prediction Based on Customer Behavior" dataset by Subham Jain, available on [Kaggle](https://www.kaggle.com/datasets/subhamjain/loan-prediction-based-on-customer-behavior). 

Dataset Statistics:
- Training set size: 252,000 records (12.3% defaults)
- Test set size: 28,000 records (12.8% defaults)
- 11 Features:
  - Demographic: Age, married, profession
  - Financial: Income, house ownership, car ownership
  - Location: City, state
  - Behavioral: Experience, current job years, current house years

Data Overview Table:

| Column | Description | Storage Type | Semantic Type | Theoretical Range | Training Data Range |
| :--- | :--- | :--- | :--- | :--- | :--- |
| Risk Flag | Defaulted on loan (0: No, 1: Yes) | Integer | Categorical (Binary) | [0, 1] | [0, 1] |
| Income | Income of the applicant | Integer | Numerical | [0, ∞] | [10K, 10M] |
| Age | Age of the applicant (in years) | Integer | Numerical | [18, ∞] | [21, 79] |
| Experience | Work experience (in years) | Integer | Numerical | [0, ∞] | [0, 20] |
| Profession | Applicant's profession | String | Categorical (Nominal) | Any profession [e.g., "Architect", "Dentist"] | 8 unique professions |
| Married | Marital status | String | Categorical (Binary) | ["single", "married"] | ["single", "married"] |
| House Ownership | Applicant owns or rents a house | String | Categorical (Nominal) | ["rented", "owned", "norent_noown"] | ["rented", "owned", "norent_noown"] |
| Car Ownership | Whether applicant owns a car | String | Categorical (Binary) | ["yes", "no"] | ["yes", "no"] |
| Current Job Years | Years in the current job | Integer | Numerical | [0, ∞] | [0, 14] |
| Current House Years | Years in the current house | Integer | Numerical | [0, ∞] | [10, 14] |
| City | City of residence | String | Categorical (Nominal) | Any city [e.g., "Mumbai", "Bangalore"] | 317 unique cities |
| State | State of residence | String | Categorical (Nominal) | Any state [e.g., "Maharashtra", "Tamil_Nadu"] | 5 unique states |

The dataset consists of three CSV files:
1. `Training Data.csv`: Contains all the features along with the target variable (`Risk Flag`) and an `ID` column. 
2. `Test Data.csv`: Contains only the features and an `ID` column from the test data.
3. `Sample Prediction Dataset.csv`: Contains the target variable (`Risk Flag`) and an `ID` column from the test data.

Example Training Data:

| Risk Flag | Income    | Age | Experience | Profession         | Married | House Ownership | Car Ownership | Current Job Years | Current House Years | City      | State         |
| :-------- | :-------- | :-- | :--------- | :----------------- | :------ | :-------------- | :------------ | :---------------- | :------------------ | :-------- | :------------ |
| 0         | 1,303,834 | 23  | 3          | Mechanical_engineer | single  | rented          | no            | 3                 | 13                   | Rewa      | Madhya_Pradesh |
| 1         | 6,256,451 | 41  | 2          | Software_Developer | single  | rented          | yes           | 2                 | 12                   | Bangalore | Tamil_Nadu    |
| 0         | 3,991,815 | 66  | 4          | Technical_writer   | married | rented          | no            | 4                 | 10                   | Alappuzha | Kerala        |

**Technical Requirements**  
The project must meet the following technical requirements:
- Data Preprocessing:
  - Load, clean, transform, and save data using `pandas` and `sklearn`.
  - Handle duplicates, incorrect data types, missing values, and outliers.
  - Extract features, scale numerical features, and encode categorical features.
- Exploratory Data Analysis (EDA):
  - Analyze descriptive statistics using `pandas` and `numpy`.
  - Visualize distributions, correlations, and relationships using `seaborn` and `matplotlib`.
- Modeling:
  - Train baseline models and perform hyperparameter tuning for binary classification task with `sklearn` and `xgboost`.
  - Baseline models: Logistic Regression, K-Nearest Neighbors, Support Vector Machine, Random Forest, Multi-Layer Perceptron, XGBoost.
  - Evaluate model performance using Area Under the Precision-Recall Curve (AUC-PR).
    - AUC-PR is more suitable to address class imbalance (12.3% defaults) with a focus on the positive class (preventing defaults) than accuracy, precision, recall, F1-score, and AUC-ROC.
    - Success criterion: Minimum AUC-PR of 0.70 on the test data.
  - Potentially use additional techniques to address class imbalance (e.g., SMOTE, class weights).
  - Visualize feature importance, show model prediction examples, and save the final model with `pickle`.
- Deployment:
  - Create a REST API for model deployment and integration with existing loan processing systems.
  - Implement efficient batch processing capabilities for multiple loan applications.
  - Deploy using cloud infrastructure to ensure scalability and security.
  - Develop monitoring systems for model performance and data drift.
- Stakeholders:
  - Loan officers: Direct users of the predictions in day-to-day loan approvals.
  - Credit risk analysts: Provide subject matter expertise on loan data defaults.
  - Compliance officers: Ensure the model adheres to any legal or regulatory guidelines.
  - IT department: Ensure infrastructure and data access for the model's development and deployment. 

By fulfilling these objectives and requirements, the project will provide a valuable tool for predicting loan defaults, thereby enhancing decision-making for financial institutions.

# Imports

In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin

# Data Loading

## CSV 
Load data from a .csv file into a Pandas DataFrame.

In [ ]:
try:
    df_train = pd.read_csv("data/training_data.csv")
    X_test = pd.read_csv("data/test_data.csv")
    y_test = pd.read_csv("data/sample_prediction_dataset.csv")
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: File not found. Please check the file path.")
except pd.errors.EmptyDataError:
    print("Error: The file is empty.")
except pd.errors.ParserError:
    print("Error: The file content could not be parsed as a CSV.")
except PermissionError:
    print("Error: Permission denied when accessing the file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# Initial Data Inspection  
Basic exploration of the dataset to understand its structure and detect obvious issues.

In [ ]:
# Show DataFrame info to check the number of rows and columns, data types and missing values
print("Training Data:")
print(df_train.info())
print("\nTest Data - Features:")
print(X_test.info())
print("\nTest Data - Target Variable:")
print(y_test.info())

In [ ]:
# Show top five rows of the training data
print("Training Data:")
df_train.head()

In [ ]:
# Show top five rows of the test data features
print("Test Data - Features:")
X_test.head()

In [ ]:
# Show top five rows of the test data target variable
print("Test Data - Target Variable:")
y_test.head()

# Data Preprocessing

## Handling Inconsistent Column Names
Some column names are inconsistent across the training and test dataset (e.g., "Id" vs. "ID", "Risk_Flag" vs. "risk_flag").  

In [ ]:
# Change column names to lowercase and replace "/" with "_"
df_train.columns = df_train.columns.str.lower().str.replace("/", "_")
X_test.columns = X_test.columns.str.lower().str.replace("/", "_")
y_test.columns = y_test.columns.str.lower().str.replace("/", "_")

## Handling Duplicates

Duplicates based on all columns:

In [103]:
# Diagnose duplicates based on all columns
print("Training Data:")
print(df_train.duplicated().value_counts())
print("\nTest Data - Features:")
print(X_test.duplicated().value_counts())
print("\nTest Data - Target Variable:")
print(y_test.duplicated().value_counts())

Training Data:
False    252000
Name: count, dtype: int64

Test Data - Features:
False    28000
Name: count, dtype: int64

Test Data - Target Variable:
False    28000
Name: count, dtype: int64


No duplicates based on all columns in training data.

Duplicates based on the ID column:

In [109]:
# Diagnose duplicates based on ID column
print("Training Data:")
print(df_train.duplicated(subset=["id"]).value_counts())
print("\nTest Data - Features:")
print(X_test.duplicated(subset=["id"]).value_counts())
print("\nTest Data - Target Variable:")
print(y_test.duplicated(subset=["id"]).value_counts())

Training Data:
False    252000
Name: count, dtype: int64

Test Data - Features:
False    28000
Name: count, dtype: int64

Test Data - Target Variable:
False    28000
Name: count, dtype: int64


There are no duplicates based on the ID column in the training data, the test data features, or the test data target variable.

## Handling Incorrect Data Types

In [111]:
# Diagnose incorrect data types
print("Training Data:")
print(df_train.dtypes)
print("\nTest Data - Features:")
print(X_test.dtypes)
print("\nTest Data - Target Variable:")
print(y_test.dtypes)

Training Data:
id                    int64
income                int64
age                   int64
experience            int64
married_single       object
house_ownership      object
car_ownership        object
profession           object
city                 object
state                object
current_job_yrs       int64
current_house_yrs     int64
risk_flag             int64
dtype: object

Test Data - Features:
id                    int64
income                int64
age                   int64
experience            int64
married_single       object
house_ownership      object
car_ownership        object
profession           object
city                 object
state                object
current_job_yrs       int64
current_house_yrs     int64
dtype: object

Test Data - Target Variable:
id           int64
risk_flag    int64
dtype: object


All the columns in the training and test data have the correct data type.

## Feature Extraction

In [122]:
# Explore number of unique categories in string columns
categorical_columns = df_train.select_dtypes(include=["object"]).columns.tolist()
print("Training Data:")
print(df_train[categorical_columns].nunique())
print("\nTest Data - Features:")
print(X_test[categorical_columns].nunique())

Training Data:
married_single       2
house_ownership      3
car_ownership        2
profession          51
city               317
state               29
dtype: int64

Test Data - Features:
married_single       2
house_ownership      3
car_ownership        2
profession          51
city               317
state               29
dtype: int64


## Handling Missing Values

In [124]:
# Diagnose missing values
print("Training Data:")
print(df_train.isnull().sum())
print("\nTest Data - Features:")
print(X_test.isnull().sum())
print("\nTest Data - Target Variable:")
print(y_test.isnull().sum())

Training Data:
id                   0
income               0
age                  0
experience           0
married_single       0
house_ownership      0
car_ownership        0
profession           0
city                 0
state                0
current_job_yrs      0
current_house_yrs    0
risk_flag            0
dtype: int64

Test Data - Features:
id                   0
income               0
age                  0
experience           0
married_single       0
house_ownership      0
car_ownership        0
profession           0
city                 0
state                0
current_job_yrs      0
current_house_yrs    0
dtype: int64

Test Data - Target Variable:
id           0
risk_flag    0
dtype: int64


There are no missing values in the training data, the test data features, or the test data target variable.

## Handling Outliers

### Remove with 3SD Method  
Remove univariate outliers from a numerical column by applying the 3 standard deviation (SD) rule. Specifically, a data point is considered an outlier if it falls more than 3 standard deviations above or below the mean of the column. 

In [130]:
# Create a custom transformer class to remove outliers using the 3SD method
class OutlierRemover3SD(BaseEstimator, TransformerMixin):
    def fit(self, df, numerical_column):
        # Calculate mean, standard deviation, and cutoff values of the numerical column
        self.mean_ = df[numerical_column].mean()
        self.sd_ = df[numerical_column].std()
        self.lower_cutoff_ = self.mean_ - 3 * self.sd_
        self.upper_cutoff_ = self.mean_ + 3 * self.sd_

        # Create a mask for filtering outliers
        self.mask_ = (df[numerical_column] >= self.lower_cutoff_) & (df[numerical_column] <= self.upper_cutoff_)

        # Show cutoff values and number of outliers
        print(f"Lower cutoff for {numerical_column}: {self.lower_cutoff_}")
        print(f"Upper cutoff for {numerical_column}: {self.upper_cutoff_}")
        print(f"{df.shape[0] - df[self.mask_].shape[0]} outliers identified based on {numerical_column} using the 3SD method.")
  
        return self

    def transform(self, df):
        # Remove outliers based on the mask 
        print(f"{df.shape[0] - df[self.mask_].shape[0]} outliers removed based on {numerical_column} using the 3SD method.")
        return df[self.mask_]

    def fit_transform(self, df, numerical_column):
        # Perform both fit and transform 
        return self.fit(df, numerical_column).transform(df)

In [132]:
# Initialize outlier remover 
outlier_remover_3sd = OutlierRemover3SD()

# Remove outliers
outlier_remover_3sd.fit(df_train, "income")

Lower cutoff for income: -3637816.3755079424
Upper cutoff for income: 13632049.706158735
0 outliers identified based on income using the 3SD method.


OutlierRemover3SD()

# Future Improvements

**Data Enrichment**:  
To enhance the model's performance and business value, data enrichment with the following financial features is recommended:
- Loan amount 
- Loan duration
- Interest rate
- Type of loan (e.g., personal, home, vehicle)
- Existing debt
- Credit score

**Enhanced Analysis Capabilities**:  
The addition of these features, particularly loan amount, would enable more precise risk assessment and better alignment with business objectives through cost-sensitive evaluation metrics incorporating actual monetary values. Cost-sensitive metrics or expected monetary value incorporate the actual cost of defaults (false negatives) and the opportunity cost of rejecting good loans (false positives).